In [ ]:
import tensorflow as tf
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import time
import pickle

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
import plotly.offline as pyo
from plotly import subplots
import plotly.graph_objects as go
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense,Input,concatenate
from tensorflow.keras.layers import *
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import lightgbm as lgb

from datetime import datetime, timedelta

from sklearn.model_selection import KFold


from baseFunctions import *
from data_helpers import processData6, featureEngineering, getSequencesFast, removeOutliers, create_sequences, dataProcessing, addLaggedFutureHolidays


In [ ]:
data, propDicts, flippedPropDicts = processData6(oneHotEnodeCatFeat=True)
data, timeFeatures = featureEngineering(data,splits=[2,2,2,2],oneHotWeekday=True)

In [ ]:
 # helps to learn function, but doesn't help with overfitting
data['linear_time'] = (data['linear_time'] - data.linear_time.mean()) /data.linear_time.std()
data['day_of_year'] = (data['day_of_year'] - data.day_of_year.mean()) /data.day_of_year.std()
data['dcoilwtico'] = (data['dcoilwtico'] - data.dcoilwtico.mean()) /data.dcoilwtico.std()
data['transactions'] = (data['transactions'] - data.transactions.mean()) /data.transactions.std()


In [ ]:
def dataProcessingLocal(storeDf, date_string_val, date_string_test, predictDiff, seasonalFDiff, seasonalLags, targetLags, rolling, initial_lag, prepPred = False):

       """ create training data based on lagged features not 2 sequences """
       trainF = [
              'store_nbr', 'family', 
              #'sales', 
              # 'dataT',
              'city', 'state', 'type', 'cluster', 
              #'dcoilwtico','onpromotion',  # added down the line!
              'holidayType',
              'description', 
              'transferred', 
              #'transactions', 
              'store_closed',
              'weekday_0', 'weekday_1', 'weekday_2',
              'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
              #'type_0','type_1','type_2','type_3','type_4', #store type
              'holidayType_0','holidayType_1','holidayType_2','holidayType_3','holidayType_4','holidayType_5','holidayType_6',
              'description_0','description_1','description_2','description_3','description_4','description_5','description_6','description_7','description_8','description_9','description_10','description_11','description_12','description_13','description_14','description_15','description_16','description_17','description_18'
              ]
       timeF = [
              'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0','day_of_year_f24_0',  'day_of_year_f52_0',
              'day_of_year_f12_180', 'day_of_year_f104_180','day_of_year_f24_180','day_of_year_f52_180', 
              #'weekday', 
              'month'
              ]
       if not prepPred:
              storeDf = storeDf.loc[(storeDf.dataT == 'train')]
              mask = (storeDf.date >= date_string_val)
              storeDf.loc[mask,['dataT']] = 'val'

       # ln tranformation
       storeDf.loc[:,['logSales']] = np.log(storeDf.sales + 1)
       storeDf.loc[:,['transactions']] = storeDf['transactions'].fillna(0)
       storeDf['onpromotion'] = (storeDf['onpromotion']- storeDf.onpromotion.mean()) /storeDf.onpromotion.std() # helps against overfitting  # 

       
       relevantSales = storeDf.loc[storeDf.dataT =='train']
       dfLen = storeDf.shape[0]


       seasonF = []
       for period in [7]: #, 14,21,28,52  104, 365
              for pf in ['logSales','transactions','dcoilwtico']:
                     dec = sm.tsa.seasonal_decompose(relevantSales[pf],period = period, model = 'additive')
                     #print(period, max(dec.seasonal))
                     f = pf+'Seasonality_'+str(period)
                     storeDf.loc[:,[f]] = addSeasonality(period, dec, dfLen)
                     seasonF.append(f)

       seasonF.append('dcoilwtico')
       seasonF.append('onpromotion')   # helps against overfitting
       #print('done with seasonal decompose')

       if predictDiff:
              storeDf.loc[:,['ref']] = storeDf['logSales'].shift(initial_lag)
              storeDf.loc[:,['target']] = storeDf['logSales'] - storeDf['ref']
       else:
              storeDf.loc[:,['target']] = storeDf['logSales']

              
       seasonDiffF2 = []
       for f in seasonF:
              newF = f+'_diff'+str(seasonalFDiff)
              storeDf.loc[:,[newF]] = storeDf[f].diff(seasonalFDiff)
              seasonDiffF2.append(newF)
       
       # seasonal featuers lags
       seasonalFLags = seasonF + seasonDiffF2
       featuresForSLag = []#trainF
       for i in seasonalLags:
              lag = i
              newF = [seasonalFLags[j] + '_lag' + str(lag) for j in range(len(seasonalFLags))]
              featuresForSLag = featuresForSLag + newF
              storeDf.loc[:,newF] = storeDf[seasonalFLags].shift(lag).to_numpy()

       seasonalFeatures = seasonF + seasonDiffF2 + featuresForSLag
       #print('done with seasonal features')
       

       
       # lag features / how many past datapoints are we tain
       featuresForLag = ['target']
       targetLagF = []#trainF
       for i in targetLags:
              lag = i+initial_lag
              newF = [featuresForLag[j] + '_lag' + str(lag) for j in range(len(featuresForLag))]
              targetLagF = targetLagF + newF
              storeDf.loc[:,newF] = storeDf[featuresForLag].shift(lag).to_numpy()
       #print('done with target lags')
       
       #------ also add future holidays!--------
       storeDf, transferredF = addLaggedFutureHolidays(storeDf, features=['transferred','holidayType'], lags=4)

       # rolling features
       createdF = seasonF + targetLagF
       rollingF = []
       for rol in rolling:
              for i in range(len(createdF)):
                     #if 'sales_t-16'  in lagF[i]:
                     #if createdF[i] in ['target','dcoilwtico']:# or 'dcoilwtico' in lagF[i]:#'target'  in lagF[i]:
                            fm = createdF[i]+'_rollingM' + str(rol)
                            fs = createdF[i]+'_rollingS' + str(rol)
                            rollingF.append(fm)
                            rollingF.append(fs)
                            storeDf.loc[:,[fm]] = storeDf[createdF[i]].rolling(rol).mean()#.copy()
                            storeDf.loc[:,[fs]] = storeDf[createdF[i]].rolling(rol).std()#.copy()
              #print('done with rolling:', rol)
       #print('done with rolling')


       allF = rollingF + timeF + trainF +transferredF +seasonalFeatures + targetLagF
       # we get a matrix that predicts only 1 timestamp -> stride it
       if len(rolling) == 0:
              storeDf = storeDf.iloc[max(max(targetLags), max(seasonalLags))+initial_lag+1:storeDf.shape[0]]
       else:
              storeDf = storeDf.iloc[max(max(targetLags), max(seasonalLags))+initial_lag+max(rolling)+1:storeDf.shape[0]]

       
       if prepPred:
              train_subDf = storeDf.loc[(storeDf.date < date_string_test) & (storeDf.dataT == 'train')]
              test_subDf  = storeDf.loc[(storeDf.date >= date_string_test)& (storeDf.dataT == 'train')]
              pred_subDf = storeDf.loc[storeDf.dataT == 'test']
       else:
              train_subDf = storeDf.loc[storeDf.date < date_string_test]
              test_subDf  = storeDf.loc[storeDf.date >= date_string_test]
              pred_subDf = storeDf.loc[storeDf.dataT =='val']
    
       return train_subDf, test_subDf, pred_subDf, allF

In [ ]:
# TODO: build more features of oil & transactions

# Problem pairs 
# family / store
# 12 / 44 -> on promotion has not enough influence!
# 13 / 26 -> lawn and garden (13) is bad for multiple stores! -> quite different dynamic from train/test -> recently more promotion, lot more sales
# 13 / 21 -> better, store closed for a while
# 12 / 37 -> different behavior in past / future

In [ ]:
""" create training data based on lagged features not 2 sequences """

initial_lag = 16 # 16 = independent from previous predictions
seasonalFDiff = 1
seasonalLags=[1,2,3,4,5,6,7,14,21] #[1,2,3,4,5,6,7,14,21,52,104]#range(21)
targetLags = [0,1,2,3,4,5,6,7,14]#[0,1,2,3,4,5,6,7,14,21,52,10]#range(21)# [7,14,21,28,35,42,52] #range(52)#[7,14,21]
rolling = [7,21] #,21,28,35,42,14,52 ,104
predictDiff =False
logTransform=True

# Date string
date_string_val = "2017-07-01"#"2017-05-01"
date_string_test = "2017-01-01"
date_string_val = "2017-07-15"#"2017-05-01"
date_string_test = "2016-07-31"

list_tr, list_te, list_val = [],[],[]
for familyId in [4]:#[6]: #data.family.unique():
       # start with only some families
       print(familyId)
       familyDf = data.loc[data.family==familyId]  

       for storeId in [40,41,42,43]:#[41]: #data.store_nbr.unique():  # when using more data we find more overfitting & more data variiety
              print('store',storeId)
              storeDf = familyDf.loc[(familyDf.store_nbr == storeId)]# & (familyDf.date > "2015-07-01")]

              train_subDf, test_subDf, val_subDf, allF = dataProcessingLocal(storeDf, date_string_val, date_string_test, predictDiff, seasonalFDiff, seasonalLags, targetLags, rolling, initial_lag, True)
              list_tr.append(train_subDf)
              list_te.append(test_subDf)
              list_val.append(val_subDf)

train_subDf = pd.concat(list_tr)
test_subDf = pd.concat(list_te)
val_subDf = pd.concat(list_val)

targetF = 'target'
if predictDiff:
       baseTrain = train_subDf['ref'].to_numpy()
       baseTest  = test_subDf['ref'].to_numpy()
       baseVal   = val_subDf['ref'].to_numpy()
else:
       baseTrain, baseTest, baseVal = [],[],[]

X_train = train_subDf[allF]#.to_numpy()
y_train = train_subDf[targetF].to_numpy()
X_test  =  test_subDf[allF]#.to_numpy()
y_test  =  test_subDf[targetF].to_numpy()  
X_val   =  val_subDf[allF]#.to_numpy()
y_val   =  val_subDf[targetF].to_numpy() 

#np.isnan(X_train).any(),np.isnan(X_test).any(),np.isnan(X_val).any(),np.isnan(y_train).any(),np.isnan(y_test).any(),np.isnan(y_val).any()
X_train.shape

In [ ]:
train_subDf[['sales','onpromotion','store_closed','date']].set_index('date').plot(), test_subDf[['sales','onpromotion','store_closed','date']].set_index('date').plot(),val_subDf[['sales','onpromotion','store_closed','date']].set_index('date').plot()

# lgbm

In [ ]:
 # compare against base lgbm that just predicts always t+16

def calcLossLGBM(pred, y, logTransform, predictDiff, base):
    if predictDiff:
        pred = np.reshape(pred, (y.shape[0])) +  np.reshape(base, (y.shape[0]))
        y = np.reshape(y, (y.shape[0])) + np.reshape(base, (y.shape[0]))

    if logTransform:
        a = np.exp(pred) -1
        y = np.exp(y) -1 
    else:
        a = (pred)
        y = (y)

    if (a < 0).any():
        a = np.clip(a, 0, 1e20)

    rmsleTrain = np.sqrt(mean_squared_log_error(a,y))
    return rmsleTrain
def plotLGBM(i, logTransform, pred, y, predictDiff, base):
    if predictDiff:
        pred = np.reshape(pred, (y.shape[0])) +  np.reshape(base, (y.shape[0]))
        y = np.reshape(y, (y.shape[0])) + np.reshape(base, (y.shape[0]))
    if logTransform:
        a = np.exp(pred[i:i+16])
        y = np.exp(y[i:i+16])
    else:
        a = (pred[i:i+16])
        y = (y[i:i+16])
    x = range(len(a))   

    fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(8, 6))
    axs.plot(x, y, color='blue',label='Original')
    axs.plot(x, a, color='red',label='pred')
    axs.set_title('index: '+str(i))
    fig.subplots_adjust(hspace=0.5)

# Set parameters for LGBM model
params = {
    'boosting':'gbdt',#'gbdt', #'rf' #'dart'
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'mse',  # Mean squared error
    'num_leaves': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    #'max_depth':10,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'force_col_wise':True,
    'num_iterations':500
}   

sample_weights = np.reshape(
    (train_subDf.onpromotion * train_subDf.onpromotion.std()) + train_subDf.onpromotion.mean()
    + 1 
    #+ train_subDf.linear_time + train_subDf.linear_time.min()
    , (-1,1)) # öffset of 1 to not have 0 weight
sample_weights = sample_weights/ max(sample_weights)

f =rollingF+timeF+trainF+lagsAndDiffs

X_train = train_subDf[f]
X_test  =  test_subDf[f]
X_val   =  val_subDf[f]

# Train the model
gbm = lgb.train(params, lgb.Dataset(X_train, label=y_train
    , weight=sample_weights
    ), valid_sets=[
    lgb.Dataset(X_test, label=y_test)
    #,lgb.Dataset(X_val, label=y_val)
    ]#,num_boost_round=100
#,callbacks=[lgb.early_stopping(stopping_rounds=100)]

)  
predtrainLGBM = gbm.predict(X_train)
predtestLGBM = gbm.predict(X_test)
predvalLGBM = gbm.predict(X_val)

print('errors: ', calcLossLGBM(predtrainLGBM, y_train, logTransform, predictDiff, base=baseTrain), calcLossLGBM(predtestLGBM, y_test, logTransform, predictDiff, baseTest))
#, calcLossLGBM(predvalLGBM, y_val, logTransform, predictDiff, baseVal))

for i in range(1):
    plotLGBM(i*16, logTransform, predtrainLGBM, y_train, predictDiff, baseTrain)
    plotLGBM(i*16, logTransform, predtestLGBM, y_test, predictDiff, baseTest)
    plotLGBM(i*16, logTransform, predvalLGBM, y_val, predictDiff, baseVal)

In [ ]:
# featuer study with lgbm

rollingF = ['logSalesSeasonality_7_rollingM7',
 'logSalesSeasonality_7_rollingS7',
 'transactionsSeasonality_7_rollingM7',
 'transactionsSeasonality_7_rollingS7',
 'dcoilwticoSeasonality_7_rollingM7',
 'dcoilwticoSeasonality_7_rollingS7',
 'logSalesSeasonality_7_rollingM21',
 'logSalesSeasonality_7_rollingS21',
 'transactionsSeasonality_7_rollingM21',
 'transactionsSeasonality_7_rollingS21',
 'dcoilwticoSeasonality_7_rollingM21',
 'dcoilwticoSeasonality_7_rollingS21',

 'dcoilwtico_rollingM7',
 'dcoilwtico_rollingS7',
 'onpromotion_rollingM7',
 'onpromotion_rollingS7',
 'target_lag16_rollingM7',
 'target_lag16_rollingS7',
 'target_lag17_rollingM7',
 'target_lag17_rollingS7',
 'target_lag18_rollingM7',
 'target_lag18_rollingS7',
 'target_lag19_rollingM7',
 'target_lag19_rollingS7',
 'target_lag20_rollingM7',
 'target_lag20_rollingS7',
 'target_lag21_rollingM7',
 'target_lag21_rollingS7',
 'target_lag22_rollingM7',
 'target_lag22_rollingS7',
 'target_lag23_rollingM7',
 'target_lag23_rollingS7',
 'target_lag30_rollingM7',
 'target_lag30_rollingS7',

 'dcoilwtico_rollingM21',
 'dcoilwtico_rollingS21',
 'onpromotion_rollingM21',
 'onpromotion_rollingS21',
 'target_lag16_rollingM21',
 'target_lag16_rollingS21',
 'target_lag17_rollingM21',
 'target_lag17_rollingS21',
 'target_lag18_rollingM21',
 'target_lag18_rollingS21',
 'target_lag19_rollingM21',
 'target_lag19_rollingS21',
 'target_lag20_rollingM21',
 'target_lag20_rollingS21',
 'target_lag21_rollingM21',
 'target_lag21_rollingS21',
 'target_lag22_rollingM21',
 'target_lag22_rollingS21',
 'target_lag23_rollingM21',
 'target_lag23_rollingS21',
 'target_lag30_rollingM21',
 'target_lag30_rollingS21']
timeF=[
 'linear_time',
 'day_of_year',
 'day_of_year_f12_0',
 'day_of_year_f104_0',
 'day_of_year_f24_0',
 'day_of_year_f52_0',
 'day_of_year_f12_180',
 'day_of_year_f104_180',
 'day_of_year_f24_180',
 'day_of_year_f52_180',
 'month']
trainF=[
 'dcoilwtico',
 'onpromotion',
 'holidayType',
 'description',
 'transferred',
 'store_closed',
 'weekday_0',
 'weekday_1',
 'weekday_2',
 'weekday_3',
 'weekday_4',
 'weekday_5',
 'weekday_6',
 'holidayType_0',
 'holidayType_1',
 'holidayType_2',
 'holidayType_3',
 'holidayType_4',
 'holidayType_5',
 'holidayType_6',
 'description_0',
 'description_1',
 'description_2',
 'description_3',
 'description_4',
 'description_5',
 'description_6',
 'description_7',
 'description_8',
 'description_9',
 'description_10',
 'description_11',
 'description_12',
 'description_13',
 'description_14',
 'description_15',
 'description_16',
 'description_17',
 'description_18']
targetLags=[
  'target_lag16',
 'target_lag17',
 'target_lag18',
 'target_lag19',
 'target_lag20',
 'target_lag21',
 'target_lag22',
 'target_lag23',
 'target_lag30']
transferredF = [
 'transferred_lag1',
 'transferred_lag-1',
 'holidayType_lag1',
 'holidayType_lag-1',
 'transferred_lag2',
 'transferred_lag-2',
 'holidayType_lag2',
 'holidayType_lag-2',
 'transferred_lag3',
 'transferred_lag-3',
 'holidayType_lag3',
 'holidayType_lag-3',
 'transferred_lag4',
 'transferred_lag-4',
 'holidayType_lag4',
 'holidayType_lag-4',
#  'transferred_lag5',
#  'transferred_lag-5',
#  'holidayType_lag5',
#  'holidayType_lag-5',
#  'transferred_lag6',
#  'transferred_lag-6',
#  'holidayType_lag6',
#  'holidayType_lag-6',
#  'transferred_lag7',
#  'transferred_lag-7',
#  'holidayType_lag7',
#  'holidayType_lag-7',
#  'transferred_lag8',
#  'transferred_lag-8',
#  'holidayType_lag8',
#  'holidayType_lag-8',
#  'transferred_lag9',
#  'transferred_lag-9',
#  'holidayType_lag9',
#  'holidayType_lag-9',
#  'transferred_lag10',
#  'transferred_lag-10',
#  'holidayType_lag10',
#  'holidayType_lag-10',
 ]

lagsAndDiffs = [
 'logSalesSeasonality_7',
 'transactionsSeasonality_7',
 'dcoilwticoSeasonality_7',
 
 'logSalesSeasonality_7_diff1',
 'transactionsSeasonality_7_diff1',
 'dcoilwticoSeasonality_7_diff1',
 'dcoilwtico_diff1',
 'onpromotion_diff1',
 'logSalesSeasonality_7_lag1',
 'transactionsSeasonality_7_lag1',
 'dcoilwticoSeasonality_7_lag1',
 'dcoilwtico_lag1',
 'onpromotion_lag1',
 'logSalesSeasonality_7_diff1_lag1',
 'transactionsSeasonality_7_diff1_lag1',
 'dcoilwticoSeasonality_7_diff1_lag1',
 'dcoilwtico_diff1_lag1',
 'onpromotion_diff1_lag1',
 'logSalesSeasonality_7_lag2',
 'transactionsSeasonality_7_lag2',
 'dcoilwticoSeasonality_7_lag2',
 'dcoilwtico_lag2',
 'onpromotion_lag2',
 'logSalesSeasonality_7_diff1_lag2',
 'transactionsSeasonality_7_diff1_lag2',
 'dcoilwticoSeasonality_7_diff1_lag2',
 'dcoilwtico_diff1_lag2',
 'onpromotion_diff1_lag2',
 'logSalesSeasonality_7_lag3',
 'transactionsSeasonality_7_lag3',
 'dcoilwticoSeasonality_7_lag3',
 'dcoilwtico_lag3',
 'onpromotion_lag3',
 'logSalesSeasonality_7_diff1_lag3',
 'transactionsSeasonality_7_diff1_lag3',
 'dcoilwticoSeasonality_7_diff1_lag3',
 'dcoilwtico_diff1_lag3',
 'onpromotion_diff1_lag3',
 'logSalesSeasonality_7_lag4',
 'transactionsSeasonality_7_lag4',
 'dcoilwticoSeasonality_7_lag4',
 'dcoilwtico_lag4',
 'onpromotion_lag4',
 'logSalesSeasonality_7_diff1_lag4',
 'transactionsSeasonality_7_diff1_lag4',
 'dcoilwticoSeasonality_7_diff1_lag4',
 'dcoilwtico_diff1_lag4',
 'onpromotion_diff1_lag4',
 'logSalesSeasonality_7_lag5',
 'transactionsSeasonality_7_lag5',
 'dcoilwticoSeasonality_7_lag5',
 'dcoilwtico_lag5',
 'onpromotion_lag5',
 'logSalesSeasonality_7_diff1_lag5',
 'transactionsSeasonality_7_diff1_lag5',
 'dcoilwticoSeasonality_7_diff1_lag5',
 'dcoilwtico_diff1_lag5',
 'onpromotion_diff1_lag5',
 'logSalesSeasonality_7_lag6',
 'transactionsSeasonality_7_lag6',
 'dcoilwticoSeasonality_7_lag6',
 'dcoilwtico_lag6',
 'onpromotion_lag6',
 'logSalesSeasonality_7_diff1_lag6',
 'transactionsSeasonality_7_diff1_lag6',
 'dcoilwticoSeasonality_7_diff1_lag6',
 'dcoilwtico_diff1_lag6',
 'onpromotion_diff1_lag6',
 'logSalesSeasonality_7_lag7',
 'transactionsSeasonality_7_lag7',
 'dcoilwticoSeasonality_7_lag7',
 'dcoilwtico_lag7',
 'onpromotion_lag7',
 'logSalesSeasonality_7_diff1_lag7',
 'transactionsSeasonality_7_diff1_lag7',
 'dcoilwticoSeasonality_7_diff1_lag7',
 'dcoilwtico_diff1_lag7',
 'onpromotion_diff1_lag7',
 'logSalesSeasonality_7_lag14',
 'transactionsSeasonality_7_lag14',
 'dcoilwticoSeasonality_7_lag14',
 'dcoilwtico_lag14',
 'onpromotion_lag14',
 'logSalesSeasonality_7_diff1_lag14',
 'transactionsSeasonality_7_diff1_lag14',
 'dcoilwticoSeasonality_7_diff1_lag14',
 'dcoilwtico_diff1_lag14',
 'onpromotion_diff1_lag14',
 'logSalesSeasonality_7_lag21',
 'transactionsSeasonality_7_lag21',
 'dcoilwticoSeasonality_7_lag21',
 'dcoilwtico_lag21',
 'onpromotion_lag21',
 'logSalesSeasonality_7_diff1_lag21',
 'transactionsSeasonality_7_diff1_lag21',
 'dcoilwticoSeasonality_7_diff1_lag21',
 'dcoilwtico_diff1_lag21',
 'onpromotion_diff1_lag21',
]
feats = [rollingF,timeF,trainF,targetLags,transferredF,lagsAndDiffs]

# Set parameters for LGBM model
params = {
    'boosting':'gbdt',#'gbdt', #'rf' #'dart'
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'mse',  # Mean squared error
    'num_leaves': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    #'max_depth':10,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'force_col_wise':True
}   

for f in feats: 
  X_train = train_subDf[f]
  X_test  =  test_subDf[f]
  X_val   =  val_subDf[f]

  # Train the model
  gbm = lgb.train(params, lgb.Dataset(X_train, label=y_train), 500, valid_sets=[lgb.Dataset(X_test, label=y_test)]
  ,callbacks=[lgb.early_stopping(stopping_rounds=500)]
  )
  predtrainLGBM = gbm.predict(X_train)
  predtestLGBM = gbm.predict(X_test)
  print('errors: ', calcLossLGBM(predtrainLGBM, y_train, logTransform, predictDiff, base=baseTrain), calcLossLGBM(predtestLGBM, y_test, logTransform, predictDiff, baseTest))
    

In [ ]:
nsplits = 5
num_iter = 500

params = {
    'boosting':'gbdt',#'gbdt', #'rf' #'dart'
    'objective': 'regression',  # Assuming you're doing regression
    'metric': 'mse',  # Mean squared error
    'num_leaves': 5,
    'lambda_l1': 0.1,
    'lambda_l2': 0.2,
    #'max_depth':10,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'force_col_wise':True,
    'num_iterations':500
}   

sample_weights = np.reshape(
    (train_subDf.onpromotion * train_subDf.onpromotion.std()) + train_subDf.onpromotion.mean()+ 1 
    #+ train_subDf.linear_time + train_subDf.linear_time.min() #makes it a lot worse
    , (-1,1)) # öffset of 1 to not have 0 weight
sample_weights = sample_weights/ max(sample_weights)

kf = KFold(n_splits=nsplits, shuffle=False)# , random_state=42) #random doesn't help
splits = kf.split(X_train,y_train)

cv_results = lgb.cv(
    params,
    lgb.Dataset(X_train, label=y_train
    #, weight=sample_weights
    ),
    num_boost_round=num_iter,
    folds=splits,
    stratified=False,  # Set to True for stratified sampling in classification
    #early_stopping_rounds=50,  # Stop if score doesn't improve for 50 rounds
    metrics=['mse'],  # Evaluation metrics to track
    seed=42,  # Set a seed for reproducibility
    return_cvbooster=True
    ,callbacks=[lgb.early_stopping(stopping_rounds=400)]
)

lenIter = len(cv_results['valid l2-mean'])
print(cv_results['valid l2-mean'][lenIter-1], cv_results['valid l2-stdv'][lenIter-1])
for i in range(nsplits):
    predtrainLGBM = cv_results['cvbooster'].boosters[i].predict(X_train)
    predtestLGBM = cv_results['cvbooster'].boosters[i].predict(X_test)

    print('errors: ', round(calcLossLGBM(predtrainLGBM, y_train, logTransform, predictDiff, base=baseTrain),3), round(calcLossLGBM(predtestLGBM, y_test, logTransform, predictDiff, baseTest),3))

In [ ]:
importance_df = (
    pd.DataFrame({
        'feature_name': gbm.feature_name(),
        'importance_gain': gbm.feature_importance(importance_type='gain'),
        'importance_split': gbm.feature_importance(importance_type='split'),
    })
    .sort_values('importance_gain', ascending=False)
    .reset_index(drop=True)
)

In [ ]:
pd.set_option('Display.max_rows', None)
importance_df

# predict everything with LGBM

In [ ]:
import dask
from dask.distributed import Client, progress,LocalCluster
from dask import delayed

In [ ]:
@delayed
def fitLGBM(storeId, family, date_string_val, date_string_test, predictDiff, seasonalFDiff, seasonalLags, targetLags, rolling, initial_lag):
       train_subDf, test_subDf, pred_subDf, allF = dataProcessing(storeId, family, date_string_val, date_string_test, predictDiff, seasonalFDiff, seasonalLags, targetLags, rolling, initial_lag, prepPred = True)
       
       targetF = 'target'
       if predictDiff:
              baseTrain = train_subDf['ref'].to_numpy()
              baseTest  = test_subDf['ref'].to_numpy()
              baseVal   = pred_subDf['ref'].to_numpy()
       else:
              baseTrain, baseTest, baseVal = [],[],[]

       X_train = train_subDf[allF].to_numpy()
       y_train = train_subDf[targetF].to_numpy()
       X_test  =  test_subDf[allF].to_numpy()
       y_test  =  test_subDf[targetF].to_numpy()  
       X_pred   =  pred_subDf[allF].to_numpy()

       params = {
           'boosting':'gbdt',#'gbdt', #'rf' #'dart'
           'objective': 'regression',  # Assuming you're doing regression
           'metric': 'mse',  # Mean squared error
           'num_leaves': 5,
           'feature_fraction': 0.9,
           'bagging_fraction': 0.8,
           'bagging_freq': 5,
           'verbose': -1,
           'force_col_wise':True,
           #'num_iterations':500
       }   

       # Train the model
       gbm = lgb.train(params, lgb.Dataset(X_train, label=y_train),500, valid_sets=[lgb.Dataset(X_test, label=y_test)],callbacks=[lgb.early_stopping(stopping_rounds=100)])
       predtrainLGBM = gbm.predict(X_train)
       predtestLGBM = gbm.predict(X_test)
       predLGBM = gbm.predict(X_pred)

       trainLoss = calcLossLGBM(predtrainLGBM, y_train, logTransform, predictDiff, base=baseTrain)
       testLoss  = calcLossLGBM(predtestLGBM, y_test, logTransform, predictDiff, baseTest)
       storeId = storeDf.store_nbr.unique()[0]
       family = storeDf.family.unique()[0]
       print('store',storeId,'family',family,'errors: ', trainLoss, testLoss,gbm.best_iteration)

       subDf = pred_subDf[['id','sales']]
       subDf.loc[:,['sales']] = np.exp(predLGBM)-1
       subDf.loc[:,['trainE']] = trainLoss
       subDf.loc[:,['testE']] = testLoss
       subDf.loc[:,['bestIt']] = gbm.best_iteration

       return subDf



In [ ]:
""" create training data based on lagged features not 2 sequences """
trainF = [
       #'store_nbr', 'family', 
       #'sales', 
       'onpromotion',# 'dataT',
       #'city', 'state', 'type', 'cluster', 
       'dcoilwtico', 
       'holidayType',
       'description', 
       'transferred', 
       'transactions', 
       'store_closed',
       'weekday_0', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6',
       #'type_0','type_1','type_2','type_3','type_4', #store type
       'holidayType_0','holidayType_1','holidayType_2','holidayType_3','holidayType_4','holidayType_5','holidayType_6',
       'description_0','description_1','description_2','description_3','description_4','description_5','description_6','description_7','description_8','description_9','description_10','description_11','description_12','description_13','description_14','description_15','description_16','description_17','description_18'

       ]
timeF = [
       'linear_time', 'day_of_year', 'day_of_year_f12_0', 'day_of_year_f104_0','day_of_year_f24_0',  'day_of_year_f52_0',
       'day_of_year_f12_180', 'day_of_year_f104_180','day_of_year_f24_180','day_of_year_f52_180', 
       #'weekday', 
       'month'
       ]

initial_lag = 16 # 16 = independent from previous predictions
seasonalFDiff = 1
seasonalLags=[1,2,3,4,5,6,7,14,21,52,104]#range(21)
targetLags = [0,1,2,3,4,5,6,7,14,21,52,10]#range(21)# [7,14,21,28,35,42,52] #range(52)#[7,14,21]
rolling = [7,14,21,28,35,42,52,104]
predictDiff =False
logTransform=True

# Date string
date_string_val = "2017-07-15"#"2017-05-01"
date_string_test = "2017-07-15"

cluster = LocalCluster(n_workers=4)
client = Client(cluster)
#cluster.scale(threads=1, memory="4GB", GPUs=-1)

results_list = []
for familyId in data.family.unique():
       # start with only some families
       print('processing',familyId)
       familyDf = data.loc[data.family==familyId]  

       for storeId in data.store_nbr.unique():
              #storeDf = familyDf.loc[(familyDf.store_nbr == storeId)] 

              result = dask.delayed(fitLGBM)(storeId, familyId, date_string_val, date_string_test, predictDiff, seasonalFDiff, seasonalLags, targetLags, rolling, initial_lag)
              results_list.append(result)
df_list = dask.compute(*results_list)

In [ ]:
pd.concat(df_list)

In [ ]:
cluster.close()

# check multiprocess output

In [ ]:
predictionsParallel = pd.read_csv("multiprocess_sarima313_1117.csv")

In [ ]:
predictionsParallel.mean()

In [ ]:
predictionsParallel['testE'].unique()

In [ ]:
predictionsParallel.set_index('id')['sales'].to_csv('sub_multiprocess_sarima.csv') # = 0.52 (vs.  0.41 sarima)

In [ ]:
# todo: find difference to sarima & investigate big errors

# correlation analysis

In [ ]:
(train_subDf[corrF].isna().sum()).any()

In [ ]:
corrF = allF + ['target']
print(len(corrF))
corr = train_subDf[corrF].corr()

In [ ]:
corr['target_lag16']

In [ ]:
a = corr['target'].dropna().sort_values()
a[0:60], a[a.shape[0]-50:a.shape[0]]

In [ ]:
a[a.shape[0]-150:a.shape[0]-100]

# NN / log regression

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
""" use a fully connected NN  for sequence with 16"""
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

tf.keras.utils.set_random_seed(42)

n_features = X_train.shape[1]

#model = Sequential()
#model.add(Dense(1,input_shape=(n_features,), activation='linear'))
#model.add(Dense(64))
#model.add(Dense(1))
input_layer = Input(shape=(n_features,))
x = BatchNormalization()(input_layer) # helps to learn function, but doesn't help with overfitting
# Define the layers and connect them
#idden_layer = Dense(16, activation='relu')(input_layer)

output_layer = Dense(1, activation='linear')(x)

# Define the model
model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='mse', optimizer='adam')

early_stopping = EarlyStopping(monitor='val_loss', patience=100)
model.fit(X_train, y_train, epochs=2000, batch_size=6400,validation_data=(X_test, y_test), callbacks=[early_stopping])

predtrain = model.predict(X_train, verbose=False)
predtest  = model.predict(X_test, verbose=False)
predvalLGBM = model.predict(X_val, verbose=False)
#baseTrain, baseTest, baseVal = [],[],[]

print('errors: ', calcLossLGBM(predtrain, y_train, logTransform, predictDiff, baseTrain), calcLossLGBM(predtest, y_test, logTransform, predictDiff, baseTest))#, calcLossLGBM(predvalLGBM, y_val, logTransform, predictDiff, baseVal))


In [ ]:
#rrors:  0.5459896492508169 0.6072244010093094 0.5857688678529015     0.3687   lags 7,14,21
#errors:  0.5412146789623853 0.6114486464169528 0.5711310694066851    0.3739
a = {}
for i,weight in enumerate(model.layers[2].get_weights()[0]):
    a[allF[i]] = weight
b = pd.DataFrame(a)
c = b.iloc[0].sort_values()
c[0:30], c[b.shape[1]-30:b.shape[1]]

In [ ]:
predtrain = model.predict(X_train, verbose=False)
predtest  = model.predict(X_test, verbose=False)
print('errors: ', calcLossLGBM(predtrain, y_train, logTransform, predictDiff, baseTrain), calcLossLGBM(predtest, y_test, logTransform, predictDiff, baseTest))

for i in range(1):
    plotLGBM(i*16, logTransform, predtrainLGBM, y_train, predictDiff, baseTrain)
    plotLGBM(i*16, logTransform, predtestLGBM, y_test, predictDiff, baseTest)

# some more feature analysis

In [ ]:
for f in allF:
    print(f)

In [ ]:
f = ['sales', 'weekday',
 #'month',
 'onpromotion',
 #'dcoilwtico',
 'holidayType',
 'description',
 'transferred',
 'store_closed',
 #'logSalesSeasonality_7',
 #'logSalesSeasonality_14',
 #'logSalesSeasonality_21',
 #'logSalesSeasonality_28',
 #'logSalesSeasonality_52',
 #'logSalesSeasonality_365'
 ]

train_subDf[f].iloc[0:50].plot()

In [ ]:
f = ['sales', #'weekday',
 #'month',
 #'onpromotion',
 'dcoilwtico',
 'dcoilwtico_diff1',
 #'holidayType',
 #'description',
 #'transferred',
 #'store_closed',
 'logSalesSeasonality_7',
 'logSalesSeasonality_14',
 #'logSalesSeasonality_21',
 #'logSalesSeasonality_28',
 #'logSalesSeasonality_52',
 #'logSalesSeasonality_365'
 ]

train_subDf[f].iloc[0:50].plot()